<a href="https://colab.research.google.com/github/abiantony123/Learning/blob/master/My_first_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple Agent design
This is part of the Modern AI Pro. The goal is to use the ReAct architecture to build a stock analysis tool that pulls ticker quotes, news and uses tools such as calculator to perform a complex human task.

In [ ]:
!pip install -U -q langchain_groq duckduckgo_search polygon

In [ ]:
from langchain_groq import ChatGroq
import os
from google.colab import userdata
llm_groq = ChatGroq(model_name="llama3-70b-8192", api_key=userdata.get("GROQ_API_KEY"))
os.environ["POLYGON_API_KEY"] = userdata.get("POLYGON_API_KEY")

In [ ]:
class Agent:
  def __init__(self,prompt):
    self.messages = [] # setting up a basic memory
    self.messages.append({"role": "system", "content": prompt})

  def __call__(self,user_message):
    self.messages.append({"role":"user","content":user_message})
    ai_response = llm_groq.invoke(self.messages).content
    self.messages.append({"role":"assistant","content":ai_response})
    return ai_response


In [ ]:
prompt = """
You operate in a loop consisting of Thought, Action, PAUSE, and Observation.
At the end of the loop, you provide an Answer.
Use Thought to describe your reasoning about the task you have been asked to complete.
Use Action to execute one of the actions available to you, then return PAUSE.
Observation will contain the result of the action.

Your available actions are:

calculate:

e.g. calculate: result=4 * 7 / 3
Runs a calculation and returns the result. Use Python, including any necessary imports, e.g., import math; math.sqrt(4).

stock_price:

e.g. stock_price: AAPL
Returns the current stock price of the given ticker symbol (e.g., AAPL for Apple Inc.).

ddgs_news:

e.g. ddgs_news: TSLA
Performs a news search for the given ticker query using DuckDuckGo and returns a summarized text of all the news articles found. Use the news content to analyze and determine whether the overall sentiment is positive, negative, or neutral.

You are responsible for retrieving the news and performing the sentiment analysis.

Example session:

Question: What is the current price of Tesla, and is the recent news about it positive or negative?
Thought: First, I need to get the current stock price of Tesla.
Action: stock_price: TSLA
PAUSE

You will be called again with this:

Observation: The current stock price of Tesla (TSLA) is $700.

Thought: Now, I should retrieve the latest news about Tesla and determine the overall sentiment.
Action: ddgs_news: Tesla
PAUSE

You will be called again with this:

Observation: Tesla secures a new battery deal with Panasonic...\nTesla's latest earnings surpass expectations...\n\n--Next News--\n\nTesla faces regulatory challenges in Europe...\n\n--Next News--\n\n...

Thought: After analyzing the news content, the sentiment appears to be mostly positive due to the strong earnings and new battery deal.
I should calcuate the number of stocks you could buy for $10,000.

Action: calculate: result=$10000/700 = 14 shares
PAUSE

Observation: You could buy 14 shares.

Answer: The current price of Tesla (TSLA) is $700. The recent news about Tesla is generally positive, with strong earnings and a new battery deal.
You could buy 14 shares to cover your investment.


If the news is neither negative or positive and you cannot make a decisiion, return a HOLD statement. IF not give a BUY or SELL statement.
This is for a lab experiment and not a trading strategy and thus be bold.

Return the number of shares that could be bought.
""".strip()

In [ ]:
from duckduckgo_search import DDGS
def search_news(query: str, region: str = 'in-en') -> str:
    try:
        news_results = DDGS().news(query, region='in-en',max_results=5)
        if not news_results:
            return f"No recent news found for {query}."

        text=""
        for article in news_results:
            text +=  article.get('title')+ "\n"+ article.get('body')+"\n\n--Next News--\n\n"

        return text

    except Exception as e:
        return f"An error occurred while searching for news: {str(e)}"

search_news("L&T Infotech")

"l t infotech\nThe two sister concerns are quite distinct in terms of business and clientele, with L&T Infotech more focussed on banking, insurance, etc, whereas Mindtree is predominant in retail and hospitality. On account of culture and values as well as work-life ...\n\n--Next News--\n\nIEX, L&T Infotech, TTK Prestige and two other stocks that have ignored the market mood to hit a one-year high\nShares of the construction conglomerate gained as much as 4% on October 19 as its subsidiary Larsen & Toubro Infotech reported strongest sequential growth in the September quarter. L&T has a 74.09 ...\n\n--Next News--\n\n'How long can you stare at wife?' L&T chief wants employees to work on Sundays\nL&T chairman SN Subrahmanyan's call for a 90-hour work week has reignited the work-life balance debate, adding to the uproar sparked by Narayana Murthy's 70-hour work week suggestion.\n\n--Next News--\n\nHappiest Minds is a smaller company that does the same thing as MindTree and L&T Infotech — b

In [ ]:
import polygon
stocks_client = polygon.StocksClient(os.getenv("POLYGON_API_KEY"))
def get_stock_price(ticker: str)->str:
    return stocks_client.get_previous_close(ticker)["results"][0]["c"]

get_stock_price("MSFT")

424.56

In [ ]:
def calculate(expression: str) -> str:
    try:
        # Set up allowed imports
        namespace = {
            'math': __import__('math'),
            'numpy': __import__('numpy')
        }
        clean_expr = expression.replace('$', '')
        # Execute in namespace
        exec(clean_expr, namespace)
        result = namespace.get('result')

        # Format result
        if isinstance(result, float):
            result = round(result, 2)

        return str(result)

    except Exception as e:
        return f"Error in calculation: {str(e)}"

# Example usage
print(calculate("import math; result = math.sqrt(4) * math.log(10)"))

4.61


In [ ]:
calculate("result=4 * 7 / 3")

'9.33'

In [ ]:
known_actions = {
    "ddgs_news": search_news,
    "stock_price": get_stock_price,
    "calculate": calculate
}

In [ ]:
def simple_query(question):
  bot = Agent(prompt)
  return bot(question)

response = simple_query("IS Tesla a good buy")
response

'Thought: To determine if Tesla is a good buy, I need to consider its current stock price and the overall sentiment of recent news about the company.\n\nAction: stock_price: TSLA\nPAUSE'

In [ ]:
import re
def parse_actions(ai_message):
  action_re = re.compile('^Action: (\w+): (.*)$')
  actions = [action_re.match(a) for a in ai_message.split('\n') if action_re.match(a)]
  print(actions)

  if actions:
        action, action_input = actions[0].groups()
        if action not in known_actions:
              raise Exception("Unknown action: {}: {}".format(action, action_input))
        print(" -- running {} {}".format(action, action_input))
        observation = known_actions[action](str(action_input))
        return observation

  return None

parse_actions(response)

[<re.Match object; span=(0, 25), match='Action: stock_price: TSLA'>]
 -- running stock_price TSLA


394.94

In [ ]:
calculate("result = 10000/214.11")

result = 10000/214.11


'46.70496473775162'

In [ ]:
def multi_step_reasoning(question, max_turns=5):
    i = 0
    bot = Agent(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)

        # Check if the result contains an 'Answer' (indicating the final step)
        if "Answer:" in result:
            print("Final Answer:", result)
            break

        observation = parse_actions(result)

        if observation is None:
            break
        print("Observation:", observation)
        next_prompt = "Observation: {}".format(observation)

# Example usage
multi_step_reasoning("Is Tesla a good buy? How many can I buy with my pension fund of $50000")

Thought: To determine if Tesla is a good buy, I need to get the current stock price of Tesla and analyze the recent news about it.

Action: stock_price: TSLA
PAUSE
[<re.Match object; span=(0, 25), match='Action: stock_price: TSLA'>]
 -- running stock_price TSLA
Observation: 394.94
Thought: Now that I have the current stock price of Tesla, I should retrieve the latest news about Tesla and determine the overall sentiment.

Action: ddgs_news: Tesla
PAUSE
[<re.Match object; span=(0, 24), match='Action: ddgs_news: Tesla'>]
 -- running ddgs_news Tesla
Observation: Tesla recalls around 239,000 vehicles over rear-view camera issue
Tesla said it was investigating to determine whether any affected vehicles have experienced stress that could result in a shorting failure

--Next News--

Tesla Model Y 2025: Price, features, specifications, and everything you need to know
Tesla has launched a revamped Model Y in China, featuring a new design and upgraded interior at a starting price of 263,500 yuan.

In [ ]:
print(multi_step_reasoning("Compare Microsoft and Apple and see which one I could buy for my portfolio now",10))

Thought: I need to compare Microsoft and Apple to determine which one is a better investment opportunity for my portfolio. I'll start by retrieving the current stock prices of both companies.

Action: stock_price: MSFT
PAUSE
[<re.Match object; span=(0, 25), match='Action: stock_price: MSFT'>]
 -- running stock_price MSFT
Observation: 424.56
Thought: Now I have the current stock price of Microsoft (MSFT). Next, I'll retrieve the current stock price of Apple (AAPL).

Action: stock_price: AAPL
PAUSE
[<re.Match object; span=(0, 25), match='Action: stock_price: AAPL'>]
 -- running stock_price AAPL
Observation: 242.7
Thought: I now have the current stock prices of both Microsoft (MSFT) and Apple (AAPL). To better understand the sentiment around these companies, I'll retrieve the latest news for each of them.

Action: ddgs_news: MSFT
PAUSE
[<re.Match object; span=(0, 23), match='Action: ddgs_news: MSFT'>]
 -- running ddgs_news MSFT
Observation: Is Microsoft Corporation (MSFT) the Best Blue Ch

In [ ]:
print(multi_step_reasoning("Compare Infosys ADR and Cognizant and see which one I could buy for my portfolio now",10))

Thought: I need to compare the current stock prices of Infosys ADR (INFY) and Cognizant (CTSH) to determine which one is a better buy for my portfolio.

Action: stock_price: INFY
PAUSE
[<re.Match object; span=(0, 25), match='Action: stock_price: INFY'>]
 -- running stock_price INFY
Observation: 22.78
Thought: Now I have the current stock price of Infosys ADR (INFY), which is $22.78. Next, I need to get the current stock price of Cognizant (CTSH).

Action: stock_price: CTSH
PAUSE
[<re.Match object; span=(0, 25), match='Action: stock_price: CTSH'>]
 -- running stock_price CTSH
Observation: 76.02
Thought: Now I have the current stock prices of both Infosys ADR (INFY) at $22.78 and Cognizant (CTSH) at $76.02. To make a fair comparison, I should retrieve the latest news about both companies and analyze the sentiment.

Action: ddgs_news: INFY
PAUSE
[<re.Match object; span=(0, 23), match='Action: ddgs_news: INFY'>]
 -- running ddgs_news INFY
Observation: Infy holds wage hike for now
New Delhi